# Web of Science
- 학술 데이터베이스 Web of Science에서 자료를 수집하는 코드 구현  
- 학내 정책상 자동화된 프로그램을 이용해서 학술 데이터에 접근하는 것은 서버 과부화를 야기하기 때문에 무분별하게 사용하면 문제발생
(사용하지 않는 것을 권고, 사용하는 경우 도서관 사이트관리자와 충분히 상의하기)  
- 작업에 앞서 본인이 학술 데이터베이스를 사용하는 과정에 대해 고민

### [작업과정]  
(1) 경희대학교 중앙도서관 접속 및 로그인  
(2) 학술 데이터베이스 로그인  
(3) 검색어 검색  
(4) 카테고리 선택  
(5) paper 수집  

In [2]:
from selenium import webdriver
from time import sleep
from random import randint
from tqdm import tqdm
import time
import pandas as pd
import sys

In [72]:
# print(sys.path)

In [9]:
sys.path.append('/Users/jiwonjang/python_tutorial/ve_env/') # 시스템 경로 추가

In [10]:
# for i in sys.path:
#     print(i)

In [11]:
import info21

In [12]:
info21ID = info21.infoID # info21 id 
# info21ID = 'id 넣어주세요!'
info21PW = info21.infoPassward # info21 password
# info21PW = '비밀번호 넣어주세요!'

# Web of Science 접속

In [13]:
### access 'web of science'
chromeDir = '/Users/jiwonjang/python_tutorial/chromedriver' # driver directory
driver = webdriver.Chrome(chromeDir)
# driver = webdriver.Chrome()
driver.implicitly_wait(1)

In [14]:
# connect khu library
univ_url = 'https://external.khu.ac.kr/sns/login.php?return_url=https%3A%2F%2Flibrary.khu.ac.kr%2Fseoul&campus=seoul'
driver.get(univ_url)
time.sleep(1)

In [15]:
# khu library login
# //*[@id="inputEmail3"]
driver.find_element_by_xpath('//*[@id="inputEmail3"]').send_keys(info21ID)
# <input type ="text" id="inputemail3"  , //*[@id="inputEmail3"]
# //*[@id="inputEmail3"]

In [16]:
driver.find_element_by_xpath('//*[@id="inputPassword3"]').send_keys(info21PW)
# //*[@id="inputPassword3"]

# 로그인 완료

In [17]:
driver.find_element_by_xpath('/html/body/div/div[1]/div/div/form/div[3]/div/button').click()
# <button type ="submit" , /html/body/div/div[1]/div/div/form/div[3]/div/button

In [18]:
# connect web of science 
driver.get('http://access.openlink.khu.ac.kr/link.n2s?url=http://isiknowledge.com/wos')

In [83]:
driver.current_url

'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/WOS_GeneralSearch_input.do?product=WOS&search_mode=GeneralSearch&SID=C1gdRfycdjNX98CkY7K&preferencesSaved='

# Query 검색

In [19]:
sid_id = driver.find_element_by_id('SID').get_attribute('value') # url 구성을 위한 sid id 생성
print('sid id:', sid_id)

sid id: C5u8QQu3tEBocVgJQdT


In [20]:
time.sleep(2)
query = 'reinforcement learning'
print(query)

reinforcement learning


In [21]:
driver.find_element_by_id('value(input1)').send_keys(query) # input query
# <input aria-role="textbox"
time.sleep(1)

In [22]:
driver.find_element_by_xpath('//*[@id="searchCell1"]/span[1]/button').click() # button click
# <button class , //*[@id="searchCell1"]/span[1]/button

# 쿼리 클릭 완료!

In [25]:
# go to the category page
WOS_category_url = 'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/RAMore.do?product=WOS&search_mode=GeneralSearch&' + sid_id + '&qid=1&ra_mode=more&ra_name=JCRCategories&colName=WOS&viewType=raMore'

In [26]:
WOS_category_url

'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/RAMore.do?product=WOS&search_mode=GeneralSearch&C5u8QQu3tEBocVgJQdT&qid=1&ra_mode=more&ra_name=JCRCategories&colName=WOS&viewType=raMore'

In [27]:
driver.get(WOS_category_url) # category 접속

In [28]:
# 관심항목 체크
driver.find_element_by_xpath('//*[@id="TASCA_2"]').click() # computer science
# <input value , //*[@id="TASCA_2"]

In [29]:
driver.find_element_by_xpath('//*[@id="TASCA_1"]').click() # neuroscience

In [30]:
# 관심항목 체크 후 하단 범위재설정 클릭
driver.find_element_by_xpath('//*[@id="raMore"]/table/tbody/tr[1]/td/table/tbody/tr/td[2]/div/button').click()
# <button class = , //*[@id="raMore"]/table/tbody/tr[6]/td/table/tbody/tr/td[1]/div/button

# category 선택 완료

In [94]:
print(driver.current_url)

http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/Search.do?product=WOS&SID=C1gdRfycdjNX98CkY7K&search_mode=GeneralSearch&prID=1aed416b-d5f2-4c35-8f12-ee2c47443545


In [31]:
# 순회할 수 있는 url을 얻기 위해 다음 페이지 선택
driver.find_element_by_xpath('//*[@id="summary_navigation"]/nav/table/tbody/tr/td[3]/a').click()
# <a class paginationnext, //*[@id="summary_navigation"]/nav/table/tbody/tr/td[3]/a

In [32]:
print(driver.current_url)

http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/summary.do?product=WOS&parentProduct=WOS&search_mode=GeneralSearch&parentQid=1&qid=2&SID=C5u8QQu3tEBocVgJQdT&colName=WOS&&update_back2search_link_param=yes&page=2


# 논문 수집

In [34]:
type(driver.current_url)

str

In [33]:
len(driver.current_url)

217

In [35]:
base_url = driver.current_url[:-1] # 기존 url 도출

In [36]:
print(base_url)

http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/summary.do?product=WOS&parentProduct=WOS&search_mode=GeneralSearch&parentQid=1&qid=2&SID=C5u8QQu3tEBocVgJQdT&colName=WOS&&update_back2search_link_param=yes&page=


In [39]:
New_post_list = []
pageNumber = 3

In [40]:
for i in range(1, pageNumber+1): 
    new_post_list_url = base_url + str(i)
    New_post_list.append(new_post_list_url)

In [41]:
New_post_list

['http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/summary.do?product=WOS&parentProduct=WOS&search_mode=GeneralSearch&parentQid=1&qid=2&SID=C5u8QQu3tEBocVgJQdT&colName=WOS&&update_back2search_link_param=yes&page=1',
 'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/summary.do?product=WOS&parentProduct=WOS&search_mode=GeneralSearch&parentQid=1&qid=2&SID=C5u8QQu3tEBocVgJQdT&colName=WOS&&update_back2search_link_param=yes&page=2',
 'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/summary.do?product=WOS&parentProduct=WOS&search_mode=GeneralSearch&parentQid=1&qid=2&SID=C5u8QQu3tEBocVgJQdT&colName=WOS&&update_back2search_link_param=yes&page=3']

In [42]:
New_post_list[0]

'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/summary.do?product=WOS&parentProduct=WOS&search_mode=GeneralSearch&parentQid=1&qid=2&SID=C5u8QQu3tEBocVgJQdT&colName=WOS&&update_back2search_link_param=yes&page=1'

In [59]:
driver.get(New_post_list[0]) # driver url 요청
sleep(randint(1,2))
paper_hrefs = driver.find_elements_by_css_selector('div.search-results-content > div:nth-child(1) > div > a')
#RECORD_11 > div.search-results-content > div:nth-child(1) > div > a

In [48]:
paper_hrefs

[<selenium.webdriver.remote.webelement.WebElement (session="71ba25ed98332002cb6a2f07faaa1362", element="594fbb46-2195-4137-a683-35519ca916c3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="71ba25ed98332002cb6a2f07faaa1362", element="c68136a3-56f7-431a-aee6-ccb600bc9fd4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="71ba25ed98332002cb6a2f07faaa1362", element="fcc9b26a-6284-4da4-8619-65d04e29f8ba")>,
 <selenium.webdriver.remote.webelement.WebElement (session="71ba25ed98332002cb6a2f07faaa1362", element="10bf2ba6-b6be-49e8-87bb-57420d04d329")>,
 <selenium.webdriver.remote.webelement.WebElement (session="71ba25ed98332002cb6a2f07faaa1362", element="71589ef3-b971-4a78-a89e-ad38d9b854b6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="71ba25ed98332002cb6a2f07faaa1362", element="96bdf2b0-f1a4-4c61-ad25-fc943c2f5eb4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="71ba25ed98332002cb6a2f07faaa1362", element="ac17e58f-1857-4395-9b17-53

In [ ]:
[]

In [49]:
len(paper_hrefs)

10

In [60]:
paper_hrefs[0]

<selenium.webdriver.remote.webelement.WebElement (session="71ba25ed98332002cb6a2f07faaa1362", element="af2c631c-966d-4ec8-acb2-b1f6e2d92ac8")>

In [52]:
paper_hrefs[0].get_attribute('href') # page 목록의 1번째 논문의 링크

'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=C5u8QQu3tEBocVgJQdT&page=1&doc=1'

In [53]:
Paper_href_list = []
for href in paper_hrefs:
    paper_title_url = href.get_attribute('href')
    Paper_href_list.append(paper_title_url)

In [54]:
Paper_href_list

['http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=C5u8QQu3tEBocVgJQdT&page=1&doc=1',
 'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=C5u8QQu3tEBocVgJQdT&page=1&doc=2',
 'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=C5u8QQu3tEBocVgJQdT&page=1&doc=3',
 'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=C5u8QQu3tEBocVgJQdT&page=1&doc=4',
 'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=C5u8QQu3tEBocVgJQdT&page=1&doc=5',
 'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=C5u8QQu3tEBocVgJQdT&page=1&doc=6',
 'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/full_record.d

### loop version

In [55]:
Paper_href_list = []
for new_url in New_post_list:
    driver.get(new_url)
    sleep(randint(1,3))
    
    # GET PAPAERS' LINKS
    paper_hrefs = driver.find_elements_by_css_selector('div.search-results-content > div:nth-child(1) > div > a')
    #RECORD_11 > div.search-results-content > div:nth-child(1) > div > a
    
    for href in paper_hrefs:
        paper_title_url = href.get_attribute('href')
        Paper_href_list.append(paper_title_url)

print('page search finish ----------------------------------------------')

page search finish ----------------------------------------------


In [58]:
len(Paper_href_list)

30

In [56]:
Paper_href_list

['http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=C5u8QQu3tEBocVgJQdT&page=1&doc=1',
 'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=C5u8QQu3tEBocVgJQdT&page=1&doc=2',
 'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=C5u8QQu3tEBocVgJQdT&page=1&doc=3',
 'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=C5u8QQu3tEBocVgJQdT&page=1&doc=4',
 'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=C5u8QQu3tEBocVgJQdT&page=1&doc=5',
 'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=C5u8QQu3tEBocVgJQdT&page=1&doc=6',
 'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/full_record.d

In [61]:
len(Paper_href_list)

30

# collect paper data 
- 각 논문에 해당하는 정보 수집

In [62]:
Paper_href_list[0]

'http://apps.webofknowledge.com.openlink.khu.ac.kr:8080/full_record.do?product=WOS&search_mode=GeneralSearch&qid=2&SID=C5u8QQu3tEBocVgJQdT&page=1&doc=1'

In [63]:
driver.get(Paper_href_list[0])

In [64]:
keyword_fetch = driver.find_elements_by_css_selector('p:nth-child(2) > a') # keywords 인덱싱
plus_keyword_fetch = driver.find_elements_by_css_selector('p:nth-child(3) > a') # plus keywords 인덱싱

In [65]:
keyword_fetch

[<selenium.webdriver.remote.webelement.WebElement (session="71ba25ed98332002cb6a2f07faaa1362", element="d97954d5-8fe6-4a00-9d5a-ac3638a53cb4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="71ba25ed98332002cb6a2f07faaa1362", element="90a9b725-45a3-42f1-a098-9196d9509fe7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="71ba25ed98332002cb6a2f07faaa1362", element="bea10d1c-1d91-471d-acbd-46ec8f7fdba0")>,
 <selenium.webdriver.remote.webelement.WebElement (session="71ba25ed98332002cb6a2f07faaa1362", element="7dab1e73-d0e6-440b-bfb8-4fcdfab1a619")>,
 <selenium.webdriver.remote.webelement.WebElement (session="71ba25ed98332002cb6a2f07faaa1362", element="5079d75a-81b0-4617-adef-5fcc597cc6ba")>,
 <selenium.webdriver.remote.webelement.WebElement (session="71ba25ed98332002cb6a2f07faaa1362", element="e62eb7c4-f1df-44f8-83d8-0523d75e6982")>,
 <selenium.webdriver.remote.webelement.WebElement (session="71ba25ed98332002cb6a2f07faaa1362", element="f6b88a46-37d3-4dfb-828c-ca

In [66]:
len(keyword_fetch)

9

In [67]:
for keyword in keyword_fetch:
    print(keyword.text)


Emotional conversation generation
Affective computing
Emotional editing
Reinforcement learning
Multitask learning





In [68]:
paper_title = driver.find_element_by_css_selector('div.title > value').text # 논문 제목 인덱싱

In [69]:
paper_title

'Emotional editing constraint conversation content generation based on reinforcement learning'

In [71]:
keyword_fetch = driver.find_elements_by_css_selector('p:nth-child(2) > a') # keywords 인덱싱
plus_keyword_fetch = driver.find_elements_by_css_selector('p:nth-child(3) > a') # plus keywords 인덱싱

keyword_list = []
for keyword in keyword_fetch:
    keyword_list.append(keyword.text)

plus_keyword_list = []
for plus_keyword in plus_keyword_fetch:
    plus_keyword_list.append(plus_keyword.text)

paper_title = driver.find_element_by_css_selector('div.title > value').text # 논문 제목 인덱싱
publish_data = driver.find_element_by_css_selector('div.block-record-info.block-record-info-source').text

In [ ]:
paper_result = {'paper_title' : paper_title, 'publish_data' : publish_data, 'keyword' : keyword_list, 'plus_keyword' : plus_keyword_list}

In [72]:
paper_result

{'paper_title': 'Emotional editing constraint conversation content generation based on reinforcement learning',
 'publish_data': 'INFORMATION FUSION\n권: 56 페이지: 70-80\nDOI: 10.1016/j.inffus.2019.10.007\n출판: APR 2020\n문서 유형:Article\nJournal Impact Factor 보기',
 'keyword': ['',
  'Emotional conversation generation',
  'Affective computing',
  'Emotional editing',
  'Reinforcement learning',
  'Multitask learning',
  '',
  '',
  ''],
 'plus_keyword': ['', '', '']}

### loop version

In [73]:
Paper_result_DF = pd.DataFrame(columns=['paper_title', 'publish_data', 'keyword', 'plus_keyword'], index=[])

In [ ]:
for url in tqdm(Paper_href_list): # tqdm 추가
    keyword_list = []
    plus_keyword_list = []
    
    sleep(randint(1,3))
    driver.get(url)
    keyword_fetch = driver.find_elements_by_css_selector('p:nth-child(2) > a') # keywords 인덱싱
    #records_form > div > div > div > div.l-column-content > div > div:nth-child(10) > p:nth-child(2) > a:nth-child(2)
    plus_keyword_fetch = driver.find_elements_by_css_selector('p:nth-child(3) > a') # plus keywords 인덱싱
    #records_form > div > div > div > div.l-column-content > div > div:nth-child(10) > p:nth-child(3) > a
    
    for keyword in keyword_fetch:
#         print(keyword.text)
        keyword_list.append(keyword.text)
        
    for plus_keyword in plus_keyword_fetch:
#         print(plus_keyword.text)
        plus_keyword_list.append(plus_keyword.text)
        
    paper_title = driver.find_element_by_css_selector('div.title > value').text # 논문 제목 인덱싱
#     print(paper_title)
    publish_data = driver.find_element_by_css_selector('div.block-record-info.block-record-info-source').text
#     print(publish_data)
    paper_result = {'paper_title' : paper_title, 'publish_data' : publish_data, 'keyword' : keyword_list, 'plus_keyword' : plus_keyword_list}
    Paper_result_DF = Paper_result_DF.append(paper_result, ignore_index=True)
    
print('finished!!!!')
# driver.close()

 27%|██▋       | 8/30 [00:39<01:50,  5.04s/it]

In [ ]:
driver.close() # driver 종료

In [68]:
Paper_result_DF.shape

(20, 4)

In [69]:
Paper_result_DF.head()

,paper_title,publish_data,keyword,plus_keyword
0,Emotional editing constraint conversation cont...,INFORMATION FUSION\n권: 56 페이지: 70-80\nDOI: 10....,"[, Emotional conversation generation, Affectiv...","[, , ]"
1,Application of deep reinforcement learning to ...,EXPERT SYSTEMS WITH APPLICATIONS\n권: 141\n논문 번...,"[, Intrusion detection, Data networks, Deep re...","[, INTERNET, SYSTEMS, NETWORK, , ]"
2,Continuous control with Stacked Deep Dynamic R...,EXPERT SYSTEMS WITH APPLICATIONS\n권: 140\n논문 번...,"[, Reinforcement learning, Policy gradient, De...","[, , ]"
3,Time-driven feature-aware jointly deep reinfor...,EXPERT SYSTEMS WITH APPLICATIONS\n권: 140\n논문 번...,"[, Algorithmic trading, Deep reinforcement lea...","[, , ]"
4,Effective image super resolution via hierarchi...,NEUROCOMPUTING\n권: 374 페이지: 109-116\nDOI: 10.1...,"[, Image super-resolution, Edges extraction, D...","[, SUPERRESOLUTION, , ]"


In [29]:
# Paper_result_DF.to_csv('WOS_example_RL.csv', header=True, encoding='utf-8')

------------------------------------------------------------------------------

# 실습!
- Web of Science 추가 작업 : 저자 이름 수집, 인용 문헌(인용 횟수) 수집
- 구글 스칼라(google scholar) 수집 프로그램 작업 (bs version, selenium version)